#### Ortam Kurulumu (Import + Proje Yolu)

Notebook `rfs/notebooks` altında olduğu için `src/` paketlerini import edebilmek adına proje kökünü `sys.path`’e ekliyoruz.
>  Notebook: .../rfs/notebooks/etl.ipynb
>  Proje kökü: .../rfs
> PROJECT_ROOT = Path.cwd().parents[0]
> sys.path.insert(0, str(PROJECT_ROOT))
> ```


In [1]:
import os
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import sys
from pathlib import Path

# etl.ipynb: .../rfs/notebooks/etl.ipynb
# proje kökü: .../rfs
PROJECT_ROOT = Path.cwd().parents[0]
sys.path.insert(0, str(PROJECT_ROOT))

# Uyarıları devre dışı bırakmak istersen açabilirsin:
# import warnings
# warnings.filterwarnings('ignore')

# Tüm sütunları göster
pd.set_option('display.max_columns', None)
# Başlıkları kırpma (tamamını göster)
pd.set_option("display.max_colwidth", None)
# Satır genişliğini artır (kırpılmayı azaltır)
pd.set_option("display.width", 2000)

# Grafik stilini ayarlayalım (isteğe bağlı)
sns.set_theme(style="whitegrid", palette="crest")
palette = sns.color_palette("crest", as_cmap=True)
plt.rcParams['figure.figsize'] = (10, 6)

#### Dosya sistemindeki en son guncel csv dosyasini yardimci fonksiyon yardimiyla okuyoruz 

In [2]:

from src.etl.loaders import load_latest_csv

latest_hb_file, hb = load_latest_csv("../data/scrapped/HB_Details_*.csv")
latest_ty_file, ty = load_latest_csv("../data/scrapped/TY_Details_*.csv")
print("Latest HB file:", latest_hb_file)
print("Latest TY file:", latest_ty_file)
print("HB shape:", hb.shape, "| TY shape:", ty.shape)

Latest HB file: ../data/scrapped/HB_Details_202512030147.csv
Latest TY file: ../data/scrapped/TY_Details_202512030228.csv
HB shape: (1793, 27) | TY shape: (1886, 27)


#### Hpsiburada EDA aşamasında görülen yüksek oranda eksik ram ssd gibi özeikleri başlık yardımıyla dolruyruyoruz trendyolda boyle bı durum olsaydı aynı fonksıyonları onun ıcın de kullanabılırdık 

In [3]:
from src.etl.title_extractors import (
    fill_column_from_title,
    extract_ram_from_title,
    extract_ssd_from_title,
    extract_refresh_rate_from_title,
    extract_screen_feature_from_title,
)

hb = fill_column_from_title(hb, "Başlık", "Ram (Sistem Belleği)", extract_ram_from_title)
hb = fill_column_from_title(hb, "Başlık", "SSD Kapasitesi", extract_ssd_from_title)
hb = fill_column_from_title(hb, "Başlık", "Ekran Yenileme Hızı", extract_refresh_rate_from_title)
hb = fill_column_from_title(hb, "Başlık", "Ekran Özelliği", extract_screen_feature_from_title)

#### Iki ayri platformdan gelen verileri birlestirerek geldikleri platformun adini ve dosya adini ekleyelim

In [4]:
def merge_ty_hb(ty: pd.DataFrame, hb: pd.DataFrame, latest_ty_file: str, latest_hb_file: str) -> pd.DataFrame:
    hb_to_ty = {
        "İşlemci": "İşlemci Modeli",
        "Maksimum İşlemci Hızı": "Maksimum İşlemci Hızı (GHz)",
        "Ram Tipi": "Ram (Sistem Belleği) Tipi",
        "Harddisk Kapasitesi": "Hard Disk Kapasitesi",
        "Max Ekran Çözünürlüğü": "Çözünürlük",
        "Ekran Özelliği": "Çözünürlük Standartı",
        "Ekran Panel Tipi": "Panel Tipi",
    }

    ty = ty.copy()
    hb = hb.copy()

    ty["Platform"], hb["Platform"] = "Trendyol", "Hepsiburada"
    ty["KaynakDosya"] = os.path.basename(latest_ty_file)
    hb["KaynakDosya"] = os.path.basename(latest_hb_file)

    cols = list(ty.columns)  # Trendyol kolon sırası = şablon

    hb = hb.rename(columns=hb_to_ty).reindex(columns=cols)
    ty = ty.reindex(columns=cols)

    return pd.concat([ty, hb], ignore_index=True)

# kullanım
df = merge_ty_hb(ty, hb, latest_ty_file, latest_hb_file)

#### Sutun adlarini ingilizceye cevirelim

In [5]:
COLS_EN = {
    "Başlık": "title",
    "Marka": "brand",
    "Kullanım Amacı": "intended_use",
    "Renk": "color",
    "Cihaz Ağırlığı": "weight",  

    "İşlemci Tipi": "cpu_family",      
    "İşlemci Modeli": "cpu_model",     
    "İşlemci Nesli": "cpu_generation",
    "İşlemci Çekirdek Sayısı": "cpu_cores",
    "Maksimum İşlemci Hızı (GHz)": "cpu_max_ghz",

    "Ram (Sistem Belleği)": "ram_gb",
    "Ram (Sistem Belleği) Tipi": "ram_type",

    "Ekran Kartı": "gpu_model",
    "Ekran Kartı Tipi": "gpu_type",
    "Ekran Kartı Hafızası": "gpu_vram_gb",
    "Ekran Kartı Bellek Tipi": "gpu_vram_type",

    "SSD Kapasitesi": "ssd_gb",
    "Hard Disk Kapasitesi": "hdd_gb",

    "Ekran Boyutu": "screen_size_inch",
    "Çözünürlük": "resolution",                 
    "Çözünürlük Standartı": "display_standard",  
    "Ekran Yenileme Hızı": "refresh_rate_hz",
    "Panel Tipi": "panel_type",

    "İşletim Sistemi": "operating_system",
    "Çekilme Zamanı": "scraped_at",
    "Fiyat (TRY)": "price_try",
    "Link": "url",
    "Platform": "platform",
    "KaynakDosya": "source_file",
}

df = df.rename(columns=COLS_EN)

In [6]:
# Eksik veri yüzdeleri
missing_percent = df.isnull().mean() * 100

# Sadece eksik verisi olanları filtrele
missing_percent = missing_percent[missing_percent > 0]

# Büyükten küçüğe sırala ve yazdır
print(missing_percent.sort_values(ascending=False))

cpu_cores           38.624626
refresh_rate_hz     18.510465
intended_use        17.477575
cpu_max_ghz         10.845338
panel_type           8.670834
display_standard     7.175863
color                7.067138
ssd_gb               3.587932
cpu_family           2.120141
weight               2.038597
ram_type             1.821147
ram_gb               1.793966
gpu_vram_type        1.603697
hdd_gb               1.386246
gpu_type             1.304702
cpu_generation       1.304702
cpu_model            1.304702
gpu_model            1.223158
operating_system     1.060071
resolution           1.032889
screen_size_inch     1.032889
gpu_vram_gb          0.951345
brand                0.679532
title                0.679532
dtype: float64


#### Buyuk kucuk harf ve bosluk duyarliligini gidermek icin normalize edelim 

In [7]:
def normalize_df_inplace(df: pd.DataFrame) -> pd.DataFrame:
    # Tüm object/string kolonlarda: strip + boşlukları tekle + küçük harf + boşları NA yap
    obj_cols = df.select_dtypes(include=["object", "string"]).columns

    df[obj_cols] = (
        df[obj_cols]
        .astype("string")
        .apply(lambda s: s.str.strip().str.replace(r"\s+", " ", regex=True).str.lower())
    )

    # common "boş/invalid" tokenları NA yap
    na_tokens = {"", "nan", "none", "null", "-", "yok", "belirtilmemiş", "belirtilmemis"}
    df[obj_cols] = df[obj_cols].mask(df[obj_cols].isin(na_tokens), pd.NA)

    return df

df = normalize_df_inplace(df)

#### NUmerik OLmayan kolonlari standartlastiralim 

##### Apple bilgisayarlar hem uretici tarafindan fiyati belirlendiginden hem de diger sistemlerle ortak ozellikleri tasimadiklarindan ve eksik veriler barindirdiklarindan droplandi

In [8]:
# Apple satırlarını kaldır + kaç satır silindiğini raporla
before = len(df)

mask_apple = (
    df["brand"].astype("string").str.contains(r"\bapple\b", case=False, na=False) |
    df["title"].astype("string").str.contains(r"\bmacbook\b|\bapple\b", case=False, na=False)
)

removed_apple = int(mask_apple.sum())
df = df.loc[~mask_apple].reset_index(drop=True)

after = len(df)
print(f"Apple filtreleme: {removed_apple} satır silindi | {before} -> {after}")


Apple filtreleme: 148 satır silindi | 3679 -> 3531


#### Tekrar eden satirlari ve basliklari silerek ayni ornegin verisetinde defalarca egitilmesinin onune gecelim 

In [9]:
def inspect_title_duplicates(
    df: pd.DataFrame,
    title_col: str = "title",
    sort_by: str = "scraped_at",
    show_top: int = 20,
    sample_rows: int = 50,
    preview_cols: list[str] | None = None,
):
    """
    Title'a göre duplicate kayıtları incelemek için özet + örnek tablo döndürür.
    Silme yapmaz.
    """
    out = df.copy()

    if sort_by in out.columns:
        out[sort_by] = pd.to_datetime(out[sort_by], errors="coerce")

    s = out[title_col].astype("string")
    dup_mask = s.duplicated(keep=False)

    dups = out.loc[dup_mask].sort_values([title_col] + ([sort_by] if sort_by in out.columns else []))

    unique_dup_titles = int(dups[title_col].astype("string").nunique())
    dup_rows = int(len(dups))
    to_drop_keep_last = int(s.duplicated(keep="last").sum())

    print("Duplicate title (benzersiz):", unique_dup_titles)
    print("Duplicate satır (toplam):", dup_rows)
    print("Silinecek satır (keep='last'):", to_drop_keep_last)

    top_counts = dups[title_col].value_counts().head(show_top).rename("count").to_frame()

    if preview_cols is None:
        preview_cols = [c for c in ["title","brand","price_try","platform","source_file","scraped_at","url"] if c in out.columns]

    sample = dups[preview_cols].head(sample_rows)

    return dups, top_counts, sample


def drop_title_duplicates(
    df: pd.DataFrame,
    title_col: str = "title",
    keep: str = "last",
    sort_by: str = "scraped_at",
) -> tuple[pd.DataFrame, int]:
    """
    Title'a göre duplicate'leri siler.
    keep='last' için scraped_at'e göre sıralayıp en yeniyi bırakır.
    """
    out = df.copy()

    if sort_by in out.columns:
        out[sort_by] = pd.to_datetime(out[sort_by], errors="coerce")
        out = out.sort_values(sort_by)

    before = len(out)
    out = out.drop_duplicates(subset=[title_col], keep=keep).reset_index(drop=True)
    removed = before - len(out)
    return out, removed

# Gormek Icin kullanım:
# dups, top_counts, sample = inspect_title_duplicates(df)
# display(top_counts)
# display(sample)
#Silmek Icin kullanım:
df, removed = drop_title_duplicates(df, keep="last")
print("Silinen satır:", removed)

Silinen satır: 147


#### Numerik Olmayan Colonlari Standartlastiralim 

In [10]:
# -------------------------
# String/Object parsers
# -------------------------
from src.etl.column_parsers import (
    parse_brand,
    parse_intended_use,
    parse_color,
    parse_weight,
    parse_cpu_family,
    parse_ram_type,
    parse_gpu_model,
    parse_gpu_type,
    parse_gpu_vram_type,
    parse_resolution,
    parse_display_standard,
    parse_panel_type,
    parse_operating_system,
)

# -------------------------
# Numeric parsers
# -------------------------
from src.etl.column_parsers import (
    parse_cpu_generation,
    parse_core_count,
    parse_max_cpu_freq,
    parse_ram_size,
    parse_gpu_memory,
    parse_capacity_gb,
    parse_screen_size,
    parse_refresh_rate,
    parse_price_try,
)

# -------------------------
# 1) STRING / OBJECT standardizasyonları
# -------------------------
df["brand"]           = df["brand"].apply(lambda x: parse_brand(x))
df["intended_use"]    = df["intended_use"].apply(lambda x: parse_intended_use(x))
df["color"]           = df["color"].apply(lambda x: parse_color(x))
df["weight"]          = df["weight"].apply(lambda x: parse_weight(x))

df["cpu_family"]      = df["cpu_family"].apply(lambda x: parse_cpu_family(x))
df["ram_type"]        = df["ram_type"].apply(lambda x: parse_ram_type(x))
df["gpu_model"]       = df["gpu_model"].apply(lambda x: parse_gpu_model(x))
df["gpu_type"]        = df["gpu_type"].apply(lambda x: parse_gpu_type(x))          
df["gpu_vram_type"]   = df["gpu_vram_type"].apply(lambda x: parse_gpu_vram_type(x))

df["resolution"]      = df["resolution"].apply(lambda x: parse_resolution(x, min_w=800, min_h=500, max_w=10000, max_h=10000))
df["display_standard"]= df["display_standard"].apply(lambda x: parse_display_standard(x))
df["panel_type"]      = df["panel_type"].apply(lambda x: parse_panel_type(x))
df["operating_system"]= df["operating_system"].apply(lambda x: parse_operating_system(x))


# -------------------------
# 2) NUMERIC/FORMAT parser'lar (min/max parametreli)
# -------------------------
# cpu_generation: HB/TY karışık; 1–15 mantıklı
df["cpu_generation"]  = df["cpu_generation"].apply(lambda x: parse_cpu_generation(x, min_gen=1, max_gen=15))

# cpu_cores: laptopta 1–24 mantıklı
df["cpu_cores"]       = df["cpu_cores"].apply(lambda x: parse_core_count(x, min_core=1, max_core=24))

# cpu_max_ghz: 1.0–6.0 GHz mantıklı
df["cpu_max_ghz"]     = df["cpu_max_ghz"].apply(lambda x: parse_max_cpu_freq(x, min_freq=1.0, max_freq=6.0))

# ram_gb: 4–256 
df["ram_gb"]          = df["ram_gb"].apply(lambda x: parse_ram_size(x, min_ram=4, max_ram=256))

# gpu_vram_gb: 0–32 (paylaşımlı=0). min_value=0 yapıyoruz ki 0 geçerli olsun
df["gpu_vram_gb"]     = df["gpu_vram_gb"].apply(lambda x: parse_gpu_memory(x, min_value=0, max_value=32, shared_value=0))

# ssd_gb / hdd_gb: 32–8192
df["ssd_gb"]          = df["ssd_gb"].apply(lambda x: parse_capacity_gb(x, min_value=32, max_value=8192))
df["hdd_gb"]          = df["hdd_gb"].apply(lambda x: parse_capacity_gb(x, min_value=32, max_value=8192))

# screen_size_inch: 7–20
df["screen_size_inch"]= df["screen_size_inch"].apply(lambda x: parse_screen_size(x, min_value=7.0, max_value=20.0))

# refresh_rate_hz: 30–360
df["refresh_rate_hz"] = df["refresh_rate_hz"].apply(lambda x: parse_refresh_rate(x, min_value=30, max_value=360))

# price_try: 1000–1000000 (gerekirse max artır)
df["price_try"]       = df["price_try"].apply(lambda x: parse_price_try(x, min_value=1000, max_value=1000000))

In [11]:
df.tail(1)

,title,brand,intended_use,color,weight,cpu_family,cpu_model,cpu_generation,cpu_cores,cpu_max_ghz,ram_gb,ram_type,gpu_model,gpu_type,gpu_vram_gb,gpu_vram_type,ssd_gb,hdd_gb,screen_size_inch,resolution,display_standard,refresh_rate_hz,panel_type,operating_system,scraped_at,price_try,url,platform,source_file
3383,v15 g4 iru 83a100a5tr i5-13420h 16 gb 512 gb ssd uhd graphics 15.6 full hd notebook,lenovo,ofis-is,siyah,2 kg ve altı,intel core i5,13420h,13.0,NaN,4.6,16.0,ddr4,intel integrated,integrated,0.0,ddr4,512.0,NaN,15.6,1920x1080,fhd,NaN,led,freedos,2025-12-03 02:28:31,24990.0,https://www.trendyol.com/lenovo/v15-g4-iru-83a100a5tr-i5-13420h-16-gb-512-gb-ssd-uhd-graphics-15-6-full-hd-notebook-p-834187887?boutiqueid=61&merchantid=114271,trendyol,ty_details_202512030228.csv


In [12]:
# Eksik veri yüzdeleri
missing_percent = df.isnull().mean() * 100

# Sadece eksik verisi olanları filtrele
missing_percent = missing_percent[missing_percent > 0]

# Büyükten küçüğe sırala ve yazdır
print(missing_percent.sort_values(ascending=False))

hdd_gb              95.094563
cpu_cores           40.455083
refresh_rate_hz     24.793144
intended_use        17.139480
panel_type          17.021277
cpu_generation      15.927896
gpu_vram_type       10.431442
cpu_max_ghz          9.604019
color                6.205674
display_standard     6.057920
cpu_model            4.078014
ram_type             3.752955
ssd_gb               3.427896
weight               3.073286
gpu_vram_gb          2.245863
cpu_family           1.713948
gpu_model            1.418440
ram_gb               1.329787
gpu_type             1.211584
operating_system     0.886525
screen_size_inch     0.738771
resolution           0.620567
title                0.029551
brand                0.029551
price_try            0.029551
dtype: float64


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3384 entries, 0 to 3383
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             3383 non-null   string        
 1   brand             3383 non-null   object        
 2   intended_use      2804 non-null   object        
 3   color             3174 non-null   object        
 4   weight            3280 non-null   object        
 5   cpu_family        3326 non-null   object        
 6   cpu_model         3246 non-null   string        
 7   cpu_generation    2845 non-null   float64       
 8   cpu_cores         2015 non-null   float64       
 9   cpu_max_ghz       3059 non-null   float64       
 10  ram_gb            3339 non-null   float64       
 11  ram_type          3257 non-null   object        
 12  gpu_model         3336 non-null   object        
 13  gpu_type          3343 non-null   object        
 14  gpu_vram_gb       3308 n

In [14]:
df.to_csv("../data/processed/laptop_data_processed.csv", index=False)